## Import

In [1]:
from pykrx import stock
from tqdm import tqdm
from datetime import datetime, timedelta

import yfinance as yf
import FinanceDataReader as fdr
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings(action='ignore')

* functions

In [2]:
def read_xlsx(name):
    instance = xw.App(visible = False)
    xlsx_data = xw.Book(name).sheets[0]
    df = xlsx_data.range('A1').options(pd.DataFrame, index = False, expand = 'table').value
    instance.quit()
    instance.kill()
    return df

def read_data(name):
    try :
        output = pd.read_excel(name)
    except ValueError:    
        output = read_xlsx(name)
    return output

def get_stock_list():
    if os.path.isfile(data_path + "df_krx.xlsx") & os.path.isfile(data_path + "df_america.xlsx"):
        df_krx = read_data(data_path + "df_krx.xlsx")
        df_america = read_data(data_path + "df_america.xlsx")
        
    else:
        df_krx = fdr.StockListing('KRX')
        df_sp500 = fdr.StockListing('S&P500')
        df_nas = fdr.StockListing('NASDAQ')
        df_nyse = fdr.StockListing('NYSE')
        df_amex = fdr.StockListing('AMEX')

        df_america = pd.concat([df_nas, df_nyse, df_amex]).drop_duplicates()
        append_code = [x for x in df_sp500.Symbol if x not in list(df_america.Symbol)]
        append_df = df_sp500.loc[df_sp500.Symbol.isin(append_code)]
        del append_df['Sector']
        df_america = pd.concat([df_america, append_df])
        df_america.index = [x for x in range(df_america.shape[0])]
        
        df_krx.to_excel(data_path + "df_krx.xlsx", index = False)
        df_america.to_excel(data_path + "df_america.xlsx", index = False)
    
    return df_krx, df_america

def change_name(symbol, df, options):
    if options:
        output = df.loc[df.Symbol == symbol, 'Name'].values[0]
    else:
        output = df.loc[df['상품번호'] == symbol, '종목명'].values[0]
    return output

def make_df(name, df):
    temp_df = pd.DataFrame(df.Close)
    temp_df.columns = ['{}'.format(name)]
    return temp_df.reset_index()

* load data

In [5]:
data_path = os.getcwd() + "\\datasets\\"

In [6]:
# 환율 가져오기
if os.path.isfile(data_path + "currency.xlsx"):
    currency = read_data(data_path + "currency.xlsx")

start_date = np.max(currency.Date)

try :
    append_curr = fdr.DataReader("USD/KRW", start = start_date)
    append_curr = append_curr.reset_index()

    currency = currency.loc[currency.Date < start_date, :]
    currency = pd.concat([currency, append_curr])
    currency.index = [x for x in range(currency.shape[0])]
    currency.to_excel(data_path + "currency.xlsx", index = False)
    
except:
    print("추가행 없음")
    pass

In [7]:
min_date = np.min(currency.Date)

In [8]:
# data 불러오기
if os.path.isfile(data_path + "df_infos.xlsx"):
    df_infos = read_data(data_path + "df_infos.xlsx")
    df_krx, df_america = get_stock_list()
    df_america = df_america.loc[:, ['Symbol', 'Name']]
    df_krx = df_krx.loc[:, ['Symbol', 'Name']]
else:
    df_krx, df_america = get_stock_list()
    df_america = df_america.loc[:, ['Symbol', 'Name']]
    df_krx = df_krx.loc[:, ['Symbol', 'Name']]
    df_infos = pd.concat([df_krx, df_america])
    df_infos = df_infos.dropna()
    df_infos.index = [x for x in range(df_infos.shape[0])]
    
    symbol_list = []

    for symbol, name in zip(df_infos.Symbol, df_infos.Name):
        if " PR " in symbol:
            symbol_list.append(re.sub(" PR ","/",symbol))
        elif (".U" in symbol) and ("Units" in name):
            symbol_list.append(re.sub(".U","/UN",symbol))
        elif "\." in symbol:
            symbol_list.append(re.sub("\.", "/", symbol))
        elif " RT" in symbol:
            symbol_list.append(symbol.split(" ")[0])
        elif symbol == 'BRKB':
            symbol_list.append("BRK/B")
        elif symbol == 'BFB':
            symbol_list.append("BF/B")
        else:
            symbol_list.append(symbol)

    df_infos['WINK_symbols'] = symbol_list
    df_infos.to_excel(data_path + "df_infos.xlsx", index = False)

In [24]:
data= read_data(data_path + "temp_data1.xlsx")
#data= read_data(data_path + "temp_data2.xls")

In [26]:
if '계좌번호' in list(data.columns):
    cate_list = ['주식', 'NYSE', 'AMEX', 'NASD']
    
    data = data.loc[data['상품구분'].isin(cate_list)]
    account = [True if "-01" in x else False for x in data['계좌번호']]
    data = data.loc[account, :]

    # 국내 주식
    stock_list = data.loc[data['상품구분'] == '주식']['종목명']
    symbol_list = list(df_infos.loc[df_infos.Name.isin(stock_list)].Symbol)
    th = len(symbol_list)

    # 미국 주식
    stock_list = data.loc[data['상품구분'] != '주식']['상품번호']
    symbol_list.extend(list(df_infos.loc[df_infos.Symbol.isin(stock_list)].Symbol))

    start_idx = 0

    for idx, symbol in enumerate(tqdm(symbol_list[start_idx:])):
        if " " in symbol:
            search_symbol = symbol
            symbol = df_infos.loc[df_infos.Symbol == symbol, :].WINK_symbols.values[0]
        else:
            search_symbol = symbol

        try:
            df = fdr.DataReader(symbol, start = min_date)

        except ValueError:
            start_idx += 1
            continue

        if df.shape[0] == 0:
            start_idx += 1
            continue

        if start_idx >= th:
            name = change_name(search_symbol, data, False)
        else:
            name = change_name(search_symbol, df_infos, True)

        if start_idx == 0 :
            temp_stock_df = make_df(name, df)
        else:
            temp_df = make_df(name, df)
            temp_stock_df = pd.merge(temp_stock_df, temp_df, on ='Date', how = 'outer')

        start_idx += 1

    temp_stock_df = temp_stock_df.sort_values("Date")
    temp_stock_df.index = [x for x in range(temp_stock_df.shape[0])]

    # 한/미 증시 시차로 인한 NA 값 발생
    if temp_stock_df.iloc[-1:].isna().sum().sum() > 0:
        temp_stock_df = temp_stock_df.iloc[:-1,:]

    # 미국 종목 환율 반영
    select_col = [0]
    select_col.extend([x for x in range(th+1, temp_stock_df.shape[1])])

    temp_df = pd.merge(temp_stock_df.iloc[:, select_col], currency.loc[:, ['Date', 'Close']], on = 'Date', how = 'inner')
    stock_df = temp_stock_df.iloc[:, :(th+1)]

    def exchange(x):
        global temp_df
        return x * temp_df['Close']

    temp_df.iloc[:, 1:-1] = temp_df.iloc[:, 1:-1].apply(exchange, axis = 0)
    stock_df = pd.merge(stock_df, temp_df, on = 'Date', how = 'inner')
    del stock_df['Close']

    stock_df.to_excel(data_path + "stock_df.xlsx", index = False)
    
else:
    data = data.iloc[1:, :]

    stock_list = data['종목명']
    symbol_list = list(df_infos.loc[df_infos.Name.isin(stock_list)].Symbol)

    start_idx = 0

    for idx, symbol in enumerate(tqdm(symbol_list[start_idx:])):
        try:
            df = fdr.DataReader(symbol, start = min_date)

        except ValueError:
            start_idx += 1
            continue

        if df.shape[0] == 0:
            start_idx += 1
            continue

        name = change_name(symbol, df_infos, True)

        if start_idx == 0 :
            stock_df = make_df(name, df)
        else:
            temp_df = make_df(name, df)
            stock_df = pd.merge(stock_df, temp_df, on ='Date', how = 'outer')

        start_idx += 1

    stock_df = stock_df.sort_values("Date")
    stock_df.index = [x for x in range(stock_df.shape[0])]

    stock_df.to_excel(data_path + "stock_df.xlsx", index = False)

100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [00:23<00:00,  1.00it/s]


In [27]:
stock_df

,Date,JW중외제약,KODEX 미국S&P500TR,KODEX 코스피TR,LG전자,NAVER,OCI,SK바이오사이언스,SK하이닉스,STX,...,카카오,카카오페이,피씨엘,한탑,애플,알파벳 A주,메타 플랫폼스(페이스북),엔비디아,퀄컴,AMD
0,1981-04-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,81.2040,NaN,NaN,NaN,NaN,3085.7520
1,1981-04-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,81.2040,NaN,NaN,NaN,NaN,3146.6550
2,1981-04-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,74.4370,NaN,NaN,NaN,NaN,3241.3930
3,1981-04-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,74.4370,NaN,NaN,NaN,NaN,3485.0050
4,1981-04-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,81.2040,NaN,NaN,NaN,NaN,3227.8590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10586,2022-09-22,18900.0,11835.0,11035.0,85300.0,206500.0,103500.0,89600.0,86000.0,3135.0,...,61300.0,57900.0,11250.0,2200.0,214546.2410,140661.6510,200612.1130,176438.0865,173727.1120,97623.1750
10587,2022-09-23,18300.0,11760.0,10850.0,84900.0,210500.0,98200.0,86200.0,83500.0,3030.0,...,61000.0,55300.0,10450.0,2005.0,214075.4287,140515.9066,199816.0669,178113.9444,172464.2771,96713.1964
10588,2022-09-26,17900.0,11665.0,10505.0,81800.0,204500.0,93300.0,84600.0,82500.0,3935.0,...,59700.0,53000.0,10200.0,1935.0,215718.7006,140459.6726,195115.4686,174955.7784,171321.5972,94860.7140
10589,2022-09-27,17650.0,11700.0,10530.0,81300.0,204500.0,94100.0,85200.0,82000.0,3625.0,...,59300.0,52600.0,10250.0,1835.0,216500.8160,139093.5000,191735.0400,177083.8580,171677.0440,95824.7220
